In [66]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import sys
from torch.utils.tensorboard import SummaryWriter

#Device Config

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#hyperparameters
input_size=784 #28*28
hidden_size=100
num_classes=10
batch_size=100
learning_rate=0.01
num_epochs=1

writer=SummaryWriter("runs/mnist")
#Dataset
train_dataset=torchvision.datasets.MNIST(root='./data',download=True,transform=transforms.ToTensor(),train=True)
test_dataset=torchvision.datasets.MNIST(root='./data',transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,shuffle=True,batch_size=batch_size)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)

examples=iter(test_loader)
example_data,example_target=next(examples)
img_grid=torchvision.utils.make_grid(example_data)
writer.add_image('mnist_images',img_grid)
writer.close()
sys.exit()
#Model 

class NeuralNet(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes):
        super(NeuralNet,self).__init__()
        self.input_size = input_size
        self.l1=nn.Linear(input_size,hidden_size)
        self.reul=nn.ReLU()
        self.l2=nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        out=self.l1(x)
        out=self.reul(out)
        out=self.l2(out)
        return out


model=NeuralNet(input_size,hidden_size,num_classes).to(device)

#loss and optimizer

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
n_total_steps=len(train_loader)
writer.add_graph(model,example_data.reshape(-1,28*28))
writer.close()

#model training
running_loss=0.0
running_correct=0
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        #100,1,20,20 reshape we need 100, 784
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        #forward
        
        outputs=model(images)
        _,predictions=torch.max(outputs,1)
        loss=criterion(outputs,labels)
        #backward
        optimizer.zero_grad()
        loss.backward
        optimizer.step()
        running_loss=loss.item()
        running_correct+=(predictions==labels).sum().item()
        
        if (i +1) %100 ==0:
            writer.add_scalar('training_loss',running_loss/100,epoch=n_total_steps+1)
            writer.add_scalar('accuracy',running_correct/100,epoch=n_total_steps+1)
            running_loss=0
            running_correct=0
            print(f'epoch {epoch +1}/{num_epochs},step {i+1}/{n_total_steps},loss={loss.item():.4f}')
#model evaluation
with torch.no_grad():
    n_correct=0
    n_samples=0
    for images,labels in test_loader:
        images=images.reshape(-1,28*28).to(device)
        labels=labels.to(device)
        outputs=model(images)
        #value,index
        _,predictions=torch.max(outputs,1)
        n_samples+=labels.shape[0]
        n_correct+=(predictions==labels).sum().item()

acc=100.0*n_correct/n_samples
print('Accuracy:',acc)

SystemExit: 

In [67]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [68]:
%tensorboard --logdir='runs'

Reusing TensorBoard on port 6007 (pid 2868), started 0:23:26 ago. (Use '!kill 2868' to kill it.)